# Dependencies

In [1]:
import pandas as pd
import numpy as np
import json
import re
from nltk.corpus import stopwords
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Importing and Cleaning Data

In [2]:
df = pd.read_json("Data_cyb.json", lines = True, orient = "columns")

ValueError: Expected object or value

In [ ]:
rating = []

for i in df["annotation"]:
    rating.append(int(i["label"][0]))
    
df["rating"] = rating

In [ ]:
df.head()

In [2]:
tweets = pd.read_csv("Test_Twitter_Comments.csv")
tweets.tail()

FileNotFoundError: [Errno 2] File b'Test_Twitter_Comments.csv' does not exist: b'Test_Twitter_Comments.csv'

In [6]:
new_df1 = df[["content", "rating"]]

In [7]:
new_df = pd.concat([new_df1,tweets])

In [8]:
X, X_test, y, y_test = train_test_split(new_df["content"], new_df["rating"], train_size = 0.8)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(X)
reviews_test_clean = preprocess_reviews(X_test)

# Baseline Model

In [10]:
baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(reviews_train_clean)
X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X_baseline, y, train_size = 0.5
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6591219997512747
Accuracy for C=0.05: 0.7024001989802263
Accuracy for C=0.25: 0.7505285412262156
Accuracy for C=0.5: 0.7679393110309662
Accuracy for C=1: 0.7818679268747668


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Has room to learn

# Remove Stop Words
Removing Stop Words

Stop words are the very common words like ‘if’, ‘but’, ‘we’, ‘he’, ‘she’, and ‘they’. We can usually remove these 

words without changing the semantics of a text 

In [11]:
from nltk.corpus import stopwords

In [12]:
english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

In [13]:
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)
X = cv.transform(no_stop_words_train)
X_test = cv.transform(no_stop_words_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6761999502611291
Accuracy for C=0.05: 0.7237005719970157
Accuracy for C=0.25: 0.769211638895797
Accuracy for C=0.5: 0.7933349912956976
Accuracy for C=1: 0.8117383735389206


### Still has room to learn

# Normalization
A common next step in text preprocessing is to normalize the words in your corpus by trying to convert all of the different forms of a given word into one. Two methods that exist for this are Stemming and Lemmatization.


# Stemming

In [14]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()

    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(stemmed_reviews_train)
X = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.682666003481721
Accuracy for C=0.05: 0.7199701566774435
Accuracy for C=0.25: 0.7684655558318826
Accuracy for C=0.5: 0.785625466301915
Accuracy for C=1: 0.8037801541904999


### Still has room to learn

# Lemmatization

In [15]:
def get_lemmatized_text(corpus):
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)
X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6781895050982343
Accuracy for C=0.05: 0.727430987316588
Accuracy for C=0.25: 0.7652325292215867
Accuracy for C=0.5: 0.7838846058194479
Accuracy for C=1: 0.7993036558070132


### Still has room to learn

# n-grams

Last time we used only single word features in our model, which we call 1-grams or unigrams. We can potentially add more predictive power to our model by adding two or three word sequences (bigrams or trigrams) as well. For example, if a review had the three word sequence “didn’t love movie” we would only consider these words individually with a unigram-only model and probably not capture that this is actually a negative sentiment because the word ‘love’ by itself is going to be highly correlated with a positive review.
The scikit-learn library makes this really easy to play around with. Just use the ngram_range argu

In [16]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 4))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_ngram = LogisticRegression(C=1)
final_ngram.fit(X, y)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_ngram.predict(X_test)))

Accuracy for C=0.01: 0.7338970405371799
Accuracy for C=0.05: 0.8331260880378015
Accuracy for C=0.25: 0.8746580452623726
Accuracy for C=0.5: 0.8796319323551356
Accuracy for C=1: 0.8786371549365829
Final Accuracy: 0.9156926137776672


### Still has room to learn

# Word Counts

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

wc_vectorizer = CountVectorizer(binary=False)
wc_vectorizer.fit(reviews_train_clean)
X = wc_vectorizer.transform(reviews_train_clean)
X_test = wc_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75, 
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6543148470529719
Accuracy for C=0.05: 0.6980850534692863
Accuracy for C=0.25: 0.7463317582690873
Accuracy for C=0.5: 0.7731907485700075
Accuracy for C=1: 0.7958219348420791


### Still has room to learn

# TF-IDF

Another common way to represent each document in a corpus is to use the tf-idf statistic (term frequency-inverse

document frequency) for each word, which is a weighting factor that we can use in place of binary or word count 

representations.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6038298930614275
Accuracy for C=0.05: 0.6391444914200448
Accuracy for C=0.25: 0.7219597115145486
Accuracy for C=0.5: 0.7463317582690873
Accuracy for C=1: 0.7746829146978363


### Still has room to learn

# Support Vector Machines (SVM)

Recall that linear classifiers tend to work well on very sparse datasets (like the one we have). Another algorithm that can produce great results with a quick training time are Support Vector Machines with a linear kernel.
Here’s an example with an n-gram range from 1 to 2:

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))

Accuracy for C=0.01: 0.8182044267595125
Accuracy for C=0.05: 0.8594876896294454
Accuracy for C=0.25: 0.8552598856005968
Accuracy for C=0.5: 0.852026858990301
Accuracy for C=1: 0.8470529718975379


In [20]:
final_svm_ngram = LinearSVC(C=0.05)
final_svm_ngram.fit(X, y)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_svm_ngram.predict(X_test)))

Final Accuracy: 0.8965431484705297


# Naive Bayes

In [21]:
new_df.to_csv("final_df.csv", index=False)

In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("basics").getOrCreate()

In [23]:
from pyspark import SparkFiles

# Load in user_data.csv from S3 into a DataFrame
spark = SparkSession.builder.master("local").appName("CsvReader").getOrCreate()
spark_df = spark.read.format("csv").option("header", "true").load(r"C:\Users\ajkim\Desktop\Final Project\final_df.csv")

In [24]:
from pyspark.sql.types import IntegerType

spark_df = spark_df.withColumn("rating1", spark_df["rating"].cast(IntegerType()))
spark_df = spark_df.drop(spark_df.rating)
spark_df = spark_df.withColumnRenamed("rating1", "rating")

In [25]:
from pyspark.sql.functions import regexp_extract, length
review_df = spark_df.withColumnRenamed("rating", "label")\
      .withColumnRenamed("content", "review_text")\
      .select(["label", "review_text"])
review_df = review_df.withColumn('review_length', length(review_df['review_text'])).dropna()
review_df.cache()

DataFrame[label: int, review_text: string, review_length: int]

In [26]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

# Create all the features to the data set
tokenizer = Tokenizer(inputCol="review_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [27]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'review_length'], outputCol='features')

In [28]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [29]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(review_df)
cleaned = cleaner.transform(review_df)

In [30]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [31]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+-----+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|         review_text|review_length|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|                  ..|            4|            [, , ..]|            [, , ..]|(262144,[30445,24...|(262144,[30445,24...|(262145,[30445,24...|[-69.720217649278...|[0.49373838023583...|       1.0|
|    0|  1  no 2  idk..?...|          234|[, , 1, , no, 2, ...|[, , 1, , 2, , id...|(262144,[4200,963...|(262144,[4200,963...|(262145,[4200,963...|[-2348.2707658471...|[1.0,7.0837981960...|       

In [32]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.804793


# Final Model

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

In [34]:
stop_words = stopwords.words('english')
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.5
)

ccc = []
c_scores = []

for c in [0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    ccc.append(c)
    c_scores.append(accuracy_score(y_val, svm.predict(X_val)))   

Accuracy for C=0.04: 0.8183061808232807
Accuracy for C=0.05: 0.8191767193135182
Accuracy for C=0.06: 0.8206690710110682
Accuracy for C=0.07: 0.8196741698793683
Accuracy for C=0.08: 0.8194254445964432
Accuracy for C=0.09: 0.8188036313891307
Accuracy for C=0.1: 0.8181818181818182
Accuracy for C=0.11: 0.8171869170501181
Accuracy for C=0.12: 0.8166894664842681
Accuracy for C=0.13: 0.8169381917671932
Accuracy for C=0.14: 0.8166894664842681
Accuracy for C=0.15: 0.8153214774281806


In [35]:
import matplotlib.pyplot as plt    
                    
plt.plot(ccc, c_scores)
plt.show()

<Figure size 640x480 with 1 Axes>

In [46]:
final = LinearSVC(tol=.000001,C=0.06)
final.fit(X, y)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final.predict(X_test)))

Final Accuracy: 0.9017657299179309


# Let's test this baby out!

In [47]:
tweets_list = [i[1]["content"] for i in tweets.iterrows()]
rating_list = [i[1]["rating"] for i in tweets.iterrows()]

In [48]:
twitter_cleaned = preprocess_reviews(tweets_list)
len(twitter_cleaned)

101

In [49]:
tws = ngram_vectorizer.transform(twitter_cleaned)
tws.shape

(101, 138965)

In [50]:
predictions = final.predict(tws[:100])

In [51]:
pd.DataFrame({"Prediction": predictions, "Actual": rating_list[:100]}).reset_index(drop=True).head()

,Prediction,Actual
0,0,0
1,0,0
2,1,1
3,0,1
4,1,1


In [52]:
total = 0
score = 0
for i, j in zip(predictions, rating_list):
    total += 1
    if i == j:
        score += 1
        
print(f"Accuracy: {score/total}")

Accuracy: 0.88


# Save & Load Model

In [53]:
from sklearn.externals import joblib

In [54]:
joblib.dump(final, "final_model_svc.pkl")

['final_model_svc.pkl']

In [55]:
try:
    retrieve_model = joblib.load("final_model_svc.pkl")
    print("using trained model")
except:
    print("model not found")
    joblib.dump(final, "final_model_svc.pkl")

using trained model
